In [33]:
# https://medium.com/towards-data-science/evaluating-rag-applications-with-ragas-81d67b0ee31a

In [34]:
import os
api_key = os.environ.get('OPENAI_API_KEY')


In [35]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders import TextLoader
from langchain.vectorstores import Chroma

from langchain.embeddings import OpenAIEmbeddings  #← OpenAIEmbeddings 가져오기
import tiktoken


In [36]:
embeddings = OpenAIEmbeddings( #← OpenAIEmbeddings를 초기화
    model="text-embedding-ada-002" #← 모델명을 지정
)

In [37]:
data_df.to_csv(TESTSET_FILE, index=False)
data_df.head()

,question,contexts,ground_truth,answer
0,"비행 차량 운전자는 어떤 교육을 받아야 하며, 그 능력을 어떻게 증명해야 하는가?",[�� 관리 기관이 정한 기준에 따라 정해진\n다.\n제5조 (운전자 교육 )\n...,비행 차량 운전자는 자동 비행 시스템과 AI 작동에 대한 적절한 교육을 받아야 하며...,비행 차량 운전자는 자동 비행 시스템과 AI 작동에 대한 적절한 교육을 받아야 하며...
1,What is required to complete the education and...,[비행자동차 운전자격 및 허가법\n제1조(목적 )\n이 법은 비행자동차의 운전 자격...,To complete the education and training program...,To complete the education and training program...
2,비행 자동차의 자동 비행과 AI 사용에 대한 기준은 무엇인가?,[비행체 자동 비행 및 AI 사용법\n제1조(목적 )\n이 법은 비행 자동차의 자동...,비행 자동차의 자동 비행과 AI 사용에 대한 기준은 안전성과 신뢰성이 확보된 자동 ...,비행 자동차의 자동 비행과 AI 사용에 대한 기준은 안전성과 신뢰성이 확보된 자동 ...
3,비행코스나 경로를 이탈한 경우 어떤 벌금이 부과되나요?,[행하는 차량에 대해서는 정상적인 비행 코스와 \n경로를 벗어나는 것이 허용된다.\...,비행코스나 경로를 이탈한 경우 150만엔 이하의 벌금이 부과됩니다.,비행코스나 경로를 이탈한 경우 150만엔 이하의 벌금이 부과됩니다.
4,What is the role of the government in regulati...,[�의 징역 또\n는 5억 원 이하의 벌금에 처한다 .\n제5조(감독 및 지도 )...,The role of the government in regulating the e...,The role of the government in regulating the e...


In [38]:

dict_data = data_df.to_dict(orient='list')

dataset = Dataset.from_dict(dict_data)

amnesty_qa = dataset
print(amnesty_qa)


Dataset({
    features: ['question', 'contexts', 'ground_truth', 'answer'],
    num_rows: 10
})


# Evaluate

### Load data from vectorDB

In [39]:
chroma_db = Chroma(  #← Chroma를 초기화
    persist_directory=CHROMA_DB_PATH,  #← 영속화 데이터 저장 위치 지정
    embedding_function=embeddings  #← 벡터화할 모델을 지정
)

retriever = chroma_db.as_retriever()


### Load test set from file

In [40]:
# data_df = pd.read_csv(TESTSET_FILE)
# dict_data = data_df.to_dict(orient='list')
# dataset = Dataset.from_dict(dict_data)
# amnesty_qa = dataset


### Create Rag Chain

In [41]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

# Define LLM
llm = ChatOpenAI(api_key=api_key,model_name="gpt-4o-mini", temperature=0)

# Define prompt template
template = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Use two sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
"""

prompt = ChatPromptTemplate.from_template(template)

# Setup RAG pipeline
rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()} 
    | prompt 
    | llm
    | StrOutputParser() 
)

In [42]:
from ragas import evaluate

from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)


# amnesty_qa = dataset
result = evaluate(
    amnesty_qa,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
)

print(result)
df = result.to_pandas()
df.to_csv(EVAL_FILE, index=False)

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

No statements were generated from the answer.


{'context_precision': 0.9000, 'faithfulness': 0.7593, 'answer_relevancy': 0.8817, 'context_recall': 0.9000}
